# kafka consumer
producer에서 생산하는 데이터를 polling하여 csv에 저장한다.<br/>
같은 시간대에 여러 rows가 들어오면(여러 sample에 대한 각각의 값) 'sample' column을 부여하여 시간대가 중복되지 않도록 한다.

In [41]:
import csv
import json
import pandas as pd
from sklearn.externals import joblib

from kafka import KafkaConsumer
from kafka.errors import KafkaError

In [2]:
rf_model = joblib.load('RF.pkl')
rf_only_env_model = joblib.load('RF_only_env.pkl')
svr_model = joblib.load('SVR.pkl')

In [100]:
#env 데이터들 duplicate 처리하기
pf = pd.read_csv('./data/seasonenv/PF_0006044.csv', index_col=0)
pf=pf.drop_duplicates(ignore_index=True)#94000->93000
pf.to_csv('./data/kafka/env/PF_0006044.csv')

## 1. hour data → hour_data_from_kafka.csv

In [24]:
fieldnames=['',
            'fatrCode',
            'facilityId',
            'measDate',
            'weeks',
            'fldCode',
            'sectCode',
            'itemCode',
            'senVal',
            'ymd',
            'cntCollect',
            'num']
predict_fieldnames=['weeks','fruitsNum', 'fruitsNumPredict', 'onlyEnv']

predict_columns_env_list=['WS','EL','CI','TE','WD','HI','TL','IR','HL','TI','RP']
predict_columns_cultivate_list=['growLength','flowerTop','stemDiameter','leavesWidth','leavesNum','flowerPosition','fruitsPosition']

In [48]:
import timeit
'저장%f'%(0.1)

'저장0.100000'

In [56]:
consumer = KafkaConsumer('hour', bootstrap_servers=['54.241.76.179:9092'])
weeks_for_checking = ['',0]
# is_header=1

for msg in consumer:
    start_time = timeit.default_timer()
    #----------hour data 저장 시작----------
    my_json = msg.value.decode('utf8').replace("'", '"')
    data = json.loads(my_json)
    
    del data['totalRows']
    del data['totalPage']
    del data['currentPage']
    
#     print(data['facilityId'], data['measDate'], data['weeks']) #poll 확인용
    
    if(is_header):
        csvfile = open(r'C:\Users\youjeongsue\Desktop\project\smartfarm-sw\smartfarm-dashboard\src\shared\data\hour_data_from_kafka.csv',
                       'w', newline='')
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(data) #여기 어딘가에서 한줄 손실?
    else:
        csvfile = open(r'C:\Users\youjeongsue\Desktop\project\smartfarm-sw\smartfarm-dashboard\src\shared\data\hour_data_from_kafka.csv',
                       'a', newline='')
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow(data)
    csvfile.close()
    end_time = timeit.default_timer()
#     print('저장: %f'%(end_time-start_time))
    #----------hour data 저장 끝----------
    
    
    #----------predict data 저장 시작----------
    #hour, week 파일 읽어오고
    end_time = timeit.default_timer()
    df = pd.read_csv(r'C:\Users\youjeongsue\Desktop\project\smartfarm-sw\smartfarm-dashboard\src\shared\data\hour_data_from_kafka.csv', index_col=0)
    df_week=pd.read_csv('./data/kafka/week/PF_0010019.csv', index_col=0)

    #업데이트 된 hour data로 평균 다시내고
    new_df=df[['weeks', 'fatrCode', 'senVal']].groupby(['weeks', 'fatrCode']).mean().reset_index()

    #아래는 예측값 추가하는 부분
    weeks=list(new_df['weeks'])[-1]
    if weeks!=weeks_for_checking[0]: #디버깅용;
        weeks_for_checking[0]==weeks
        weeks_for_checking[1]=0
    else:
        weeks_for_checking[1]=1
        
    target_df=new_df[new_df['weeks']==weeks]

    predict_result=0 #모델 결과
    only_env=0 #생장정보가 없는지 여부 -> 다른 모델 사용
    predict_columns_list=[] #모델에 사용할 columns, 순서에 맞춰서 append 해준다.
    #현재 주차의 환경변수 평균값
    predict_columns_dict=target_df[['fatrCode','senVal']].set_index('fatrCode').to_dict()['senVal']

    #예측값 계산(생육정보가 없는경우/있는경우)->모델에 필요한 column에 맞춰 가공
    if weeks not in list(df_week['weeks']):
        only_env=1
        for code in predict_columns_env_list:
            try:
                predict_columns_list.append(predict_columns_dict[code])
            except:
                predict_columns_list.append(0) #가끔 센서값이 안들어오는 경우가 있음. 일단 그냥 0처리,,

        predict_result=rf_only_env_model.predict([predict_columns_list])[0]
    else:
#         if weeks_for_checking[1]==0:
#             print(weeks) #디버깅용;
            
        for code in predict_columns_env_list:
            try:
                predict_columns_list.append(predict_columns_dict[code])
            except:
                predict_columns_list.append(0)

        df_with_weeks=df_week[df_week['weeks']==weeks]
        for code in predict_columns_cultivate_list:
            try:
                if(math.isnan(list(df_with_weeks[code])[0])):
                    predict_columns_list.append(0)
                else:
                    predict_columns_list.append(list(df_with_weeks[code])[0])
            except:
                predict_columns_list.append(0)

        try:
            predict_result=rf_model.predict([predict_columns_list])[0]
        except:
            pass #Input contains NaN, infinity or a value too large for dtype('float32'). -> None 처리 했는데 에러나는 이유는..?

    #예측값이 없는 경우(len==0) 새로 추가, 있으면 수정
    #다 새로추가하고 fastapi 단에서 drop_duplicat(last)해서 주자..
    #header만 위에처럼 처리
    
    #실제 기록된 생산량이 있으면 data에 입력하고, 없으면 None
    fruitsNum=0
    if(len(df_week[df_week['weeks']==weeks])!=0):
        fruitsNum=list(df_week.loc[df_week['weeks']==weeks, 'fruitsNum'])[0]
    else:
        fruitsNum=None
        
    data={'weeks': weeks, 'fruitsNum': fruitsNum,'fruitsNumPredict': predict_result, 'onlyEnv': only_env}
    if(is_header):
        csvfile = open(r'C:\Users\youjeongsue\Desktop\project\smartfarm-sw\smartfarm-dashboard\src\shared\data\predict_data_from_kafka.csv',
                       'w', newline='')
        writer = csv.DictWriter(csvfile, fieldnames=predict_fieldnames)
        writer.writeheader()
        writer.writerow(data)
    else:
        csvfile = open(r'C:\Users\youjeongsue\Desktop\project\smartfarm-sw\smartfarm-dashboard\src\shared\data\predict_data_from_kafka.csv',
                       'a', newline='')
        writer = csv.DictWriter(csvfile, fieldnames=predict_fieldnames)
        writer.writerow(data)
    csvfile.close()
    #----------predict data 저장 끝----------
    end_time = timeit.default_timer()
#     print('예측 처리: %f'%(end_time-start_time))
    
    #header 초기화
    is_header=0
    

KeyboardInterrupt: 

In [55]:
rf_model.predict([predict_columns_list])[0]

30.652623337446872

## 2. week data → week_data_from_kafka.csv

In [23]:
fieldnames=['userId','measDate','weeks','sample','WS','EL','CI','TE','WD','HI','TL','IR','HL','TI','RP',
            'index','growLength','flowerTop','stemDiameter','leavesLength',
            'leavesWidth','leavesNum','flowerPosition','fruitsPosition',
            'fruitsNum','fruitsNum_predict','leavesLength_predict']

In [24]:
consumer = KafkaConsumer('week', bootstrap_servers=['54.241.76.179:9092'])

is_header=1
user, measDate = None, None
sample=1

for msg in consumer:
    my_json = msg.value.decode('utf8').replace("'", '"')
    data = json.loads(my_json)
    del data['']
    
    #start to sample 전처리
    if((user, measDate) != (data['userId'], data['measDate'])):
        user, measDate = data['userId'], data['measDate']
        sample=1
        data['sample']=sample
    else:
        sample+=1
        data['sample']=sample
    
    print(data['userId'], data['measDate'], data['sample']) #poll 확인용
    #end to sample 전처리
    
    if(is_header):
        csvfile = open(r'C:\Users\youjeongsue\Desktop\project\smartfarm-sw\smartfarm-dashboard\src\shared\data\week_data_from_kafka.csv',
                       'w', newline='')
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(data)
        is_header=0
    else:
        csvfile = open(r'C:\Users\youjeongsue\Desktop\project\smartfarm-sw\smartfarm-dashboard\src\shared\data\week_data_from_kafka.csv',
                       'a', newline='')
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow(data)

KeyError: 'userId'